In [ ]:
import pandas as pd


def Project_time_report(directory):
    #df = pd.read_csv(".../Tank_Process.csv")
    #df = pd.read_csv("xxx.csv")
    df = pd.read_csv(directory)
    labels = ["Batch_Utilizated_Time", "Total_Scheduled_Batch","Total_Scheduled_Time"]
    df = df.drop(labels,axis = 1)
    return df

def Calculation_by_Project(df, fileSelection=1):
    def runtime_cal(Data, startTime, endTime, levels = False):
        
        
        DetailDict = dict() # attribute of equipment ex: running time
        timeList = list() # Time column in Data
        statusList = list() # status column in Data
        status = 0 # status
        recodTime = 0 # recording temp time
        runningTime = 0
        idelTIme = 0
        normal = 1
        
        for i in range(len(Data)):
            if Data.iat[i, 0] != ' ' or Data.iat[i, 1] != ' ':
                try:
                    
                    if float(Data.iat[i, 0]) >= startTime and float(Data.iat[i, 0]) <= endTime:
                        timeList.append(Data.iat[i, 0])
                        statusList.append(Data.iat[i,1])
                except ValueError: 
                    timeList.append(Data.iat[i, 0])
                    statusList.append(Data.iat[i,1])
        #print(statusList)
        
        
        
        if type(timeList[0]) == str:
            try:
                if type(float(timeList[0])) == float:
                    normal = 1
            except ValueError:
                normal = 0
                DetailDict["idelTime"] = timeList[0]
                DetailDict["totalTime"] = timeList[0]
                DetailDict["runningTime"] = timeList[0]
                DetailDict["Utilization"] = timeList[0]
        
        if normal == 1:
            if levels == False:

                if statusList[0] == "RUNNING" or  statusList[0] == "1":
                    status = 1
                    recodTime = float(timeList[0])
                else:
                    status = 0
                for j in range(len(statusList)):
                    if statusList[j] == "RUNNING" or statusList[j] == "1":
                        if status == 0:
                            recodTime = float(timeList[j])
                            status = 1
                        else:
                            pass
                    elif statusList[j] == "STOPPED" or statusList[j] == "0":
                        if status == 0:
                            pass
                        else:
                            runningTime = runningTime + (float(timeList[j]) - recodTime)*24
                            status = 0

                if statusList[len(statusList)-1] == "RUNNING" or statusList[len(statusList)-1] == "1":
                    runningTime = (float(timeList[len(statusList)-1]) - recodTime)*24 + runningTime
            elif levels == True:
                
                try:
                    
                    if float(statusList[0]) > 1.00:
                        status = 1
                        recodTime = float(timeList[0])
                    else:
                        status = 0

                    for j in range(len(statusList)):

                        try:                        
                            if float(statusList[j]) > 1.00:
                                if status == 0:
                                    recodTime = float(timeList[j])
                                    status = 1
                                    #print(">")
                                else:
                                    pass
                            elif float(statusList[j]) < 1.00:
                                if status == 0:
                                    pass
                                else:
                                    runningTime = runningTime + (float(timeList[j]) - recodTime)*24
                                    status = 0
                        except ValueError:
                            pass
                    if float(statusList[len(statusList)-1]) > 1.00:
                        runningTime =  (float(timeList[len(statusList)-1]) - recodTime)*24 + runningTime
                except ValueError:
                    pass

            idelTime = (float(timeList[len(timeList)-1])  - float(timeList[0]))*24 - runningTime
            totalTime = (float(timeList[len(timeList)-1]) - float(timeList[0]))*24


            DetailDict["idelTime"] = idelTime
            DetailDict["totalTime"] = totalTime
            DetailDict["runningTime"] = runningTime
            DetailDict["Utilization"] = runningTime/totalTime
            

        return DetailDict
    
    
    def TransformTimeFormat(Data):
        from datetime import date

        timeList = list()
        for i in range(len(Data)):
            if Data.iat[i, 0] != ' ' or Data.iat[i, 1] != ' ':
                try:    
                    timeList.append(Data.iat[i, 0])



                except ValueError: 
                    timeList.append(Data.iat[i, 0])

        #print(timeList)
        d0 = date(1900, 1, 1)

        new_timeList = list()# handle Transfomred Time Format
        for k in range(len(timeList)):
            year = timeList[k].year
            month = timeList[k].month
            day = timeList[k].day
            hour = timeList[k].hour
            minute = timeList[k].minute
            second = timeList[k].second


            d1 = date(year, month, day)
            delta = d1 - d0
            new_timeList. append(delta.days+2+hour/24+minute/24/60+second/24/60/60)
            #print(delta.days+2+hour/24+minute/24/60+second/24/60/60)
        return new_timeList

    TankList = list()
    ProcessList = list()
    StartTimeList = list()
    FinishTimeList = list()
    for k in range(len(df)):
        TankList.append(df.iat[k, 0][2:])
        ProcessList.append(df.iat[k, 1])
        StartTimeList.append(df.iat[k, 2])
        FinishTimeList.append(df.iat[k,3])


    Processdict = dict()
    for j in range(len(TankList)):
        if ProcessList[j] not in Processdict:
            emptylist = list()
            Processdict[ProcessList[j]] = emptylist

    Tankdcit = dict() # overall Tank
    for i in range(len(TankList)):
        try:
            Filename  = "C:/Users/johnn/Python/Industry Practicum/new data/" + "Tank " + str(TankList[i]) + ".csv"
            data = pd.read_csv(Filename)

            agi = data[["AgiTime", "Agitator Status"]]
            level = data[["LevelTime", "Tank Level"]]
            jack = data[["JackPTime", "Tank Jacket Pump"]]
            samp = data[["SamPTime", "Tank Sample Pump"]]
            con = data[["ConPTime", "Condenser Circulation Pump"]]

            agi = agi.dropna()
            level = level.dropna()
            jack = jack.dropna()
            samp = samp.dropna()
            con = con.dropna()


            # Handle different time format

            try:
                if float(agi.iat[0,0]) > 4000:
                    pass

            except:
                try:
                    agi['AgiTime'] = pd.to_datetime(agi['AgiTime'])
                    agi['AgiTime'] = pd.DataFrame(TransformTimeFormat(agi))
                except:
                    pass
            try:
                if float(level.iat[0,0]) > 4000:
                    pass
            except:
                try:
                    level['LevelTime'] = pd.to_datetime(level['LevelTime'])
                    level['LevelTime'] = pd.DataFrame(TransformTimeFormat(level))
                except:
                    pass

            try:
                if float(jack.iat[0,0]) > 4000:
                    pass
            except:
                try:
                    jack['JackPTime'] = pd.to_datetime(jack['JackPTime'])
                    jack['JackPTime'] = pd.DataFrame(TransformTimeFormat(jack))
                except:
                    pass 

            try:    
                if float(samp.iat[0,0]) > 4000:
                    pass
            except:
                try:
                    samp['SamPTime'] = pd.to_datetime(samp['SamPTime'])
                    samp['SamPTime'] = pd.DataFrame(TransformTimeFormat(samp))
                except:
                    pass
            try:    
                if float(con.iat[0,0]) > 4000:
                    pass
            except:
                try:
                    con['ConPTime'] = pd.to_datetime(con['ConPTime'])
                    con['ConPTime'] = pd.DataFrame(TransformTimeFormat(con))
                except:
                    pass

            TankEquipDict = dict() # tank equipment Dict ex: agitator, level

            TankEquipDict["Agitator"] = runtime_cal(agi, StartTimeList[i], FinishTimeList[i])
            TankEquipDict["TankLevel"] = runtime_cal(level, StartTimeList[i], FinishTimeList[i], levels = True)
            TankEquipDict["JacketPump"] = runtime_cal(jack, StartTimeList[i], FinishTimeList[i])
            TankEquipDict["SamplePump"] = runtime_cal(samp, StartTimeList[i], FinishTimeList[i])
            TankEquipDict["CondenserPump"] = runtime_cal(con, StartTimeList[i], FinishTimeList[i])

            Tankdcit = dict()
            Tankdcit[TankList[i]] = TankEquipDict

            Processdict[ProcessList[i]].append(Tankdcit)
        except:

            pass
    if fileSelection == 1:
        
        Colname = ["Tank No","Project", "Agitator Running Time", "Agitator Idle Time", "Agitator Utilization Rate", "Tank Level Running Time", "Tank Level Idle Time", "Tank Level Utilization Rate",
              "JacketPump Running Time", "JacketPump idle Time", "JacketPump Utilization Rate", "SamplePump Running Time", "SamplePump Idle Time", "SamplePump Utilization Rate",
              "CondenserPump Running Time", "CondenserPump Idle Time", "CondenserPump Utilization Rate"]
        report_file = pd.DataFrame(columns = Colname )
        for i in Processdict:
            for j in range(len(Processdict[i])):
                TempLit = list()
                for key in Processdict[i][j].keys():
                    TempLit.append(key)
                    k = key
                    Project = i
                    TempLit.append(Project)
                    TempLit.append(Processdict[i][j][k]['Agitator']['runningTime'])
                    TempLit.append(Processdict[i][j][k]['Agitator']['idelTime'])
                    TempLit.append(Processdict[i][j][k]['Agitator']['Utilization'])
                    TempLit.append(Processdict[i][j][k]['TankLevel']['runningTime'])
                    TempLit.append(Processdict[i][j][k]['TankLevel']['idelTime'])
                    TempLit.append(Processdict[i][j][k]['TankLevel']['Utilization'])
                    TempLit.append(Processdict[i][j][k]['JacketPump']['runningTime'])
                    TempLit.append(Processdict[i][j][k]['JacketPump']['idelTime'])
                    TempLit.append(Processdict[i][j][k]['JacketPump']['Utilization'])
                    TempLit.append(Processdict[i][j][k]['SamplePump']['runningTime'])
                    TempLit.append(Processdict[i][j][k]['SamplePump']['idelTime'])
                    TempLit.append(Processdict[i][j][k]['SamplePump']['Utilization'])
                    TempLit.append(Processdict[i][j][k]['CondenserPump']['runningTime'])
                    TempLit.append(Processdict[i][j][k]['CondenserPump']['idelTime'])
                    TempLit.append(Processdict[i][j][k]['CondenserPump']['Utilization'])
                    c = pd.Series(TempLit, index = Colname)
                report_file = report_file.append(c, ignore_index=True)
    else:
        Colname_2 = ['Tank no', 'Project', 'Equipmnet', 'Idel Time', 'Total Time', 'Running Time', 'Utilization']
        report_file = pd.DataFrame(columns = Colname_2 )
        for i in Processdict:
            for j in range(len(Processdict[i])):

                for key in Processdict[i][j].keys():

                    for a in Processdict[i][j][key].keys():
                        TempLit = list()
                        TempLit.append(key)
                        k = key
                        Project = i
                        TempLit.append(Project)
                        TempLit.append(a)
                        TempLit.append(Processdict[i][j][key][a]['idelTime'])
                        TempLit.append(Processdict[i][j][key][a]['totalTime'])
                        TempLit.append(Processdict[i][j][key][a]['runningTime'])
                        TempLit.append(Processdict[i][j][key][a]['Utilization'])
                        #print(TempLit)    
                        c = pd.Series(TempLit, index = Colname_2)
                        #print(c)
                        report_file = report_file.append(c, ignore_index=True)
    return report_file


def output_report(directory, file):
    import pandas as pd
    directory = directory + '/Utilzation Report by Project.csv'
    file.to_csv(directory)



               
